# 37. Plot and summarize MWAS stage 2 outputs

This actualyl comes right before notebook 40

## Test for one

In [1]:
library(data.table)
library(parallel)
library(qqman)
library(ggplot2)
library(stringr)
library(foreach)
library(doParallel)



For example usage please run: vignette('qqman')



Citation appreciated but not required:

Turner, (2018). qqman: an R package for visualizing GWAS results using Q-Q and manhattan plots. Journal of Open Source Software, 3(25), 731, https://doi.org/10.21105/joss.00731.



Loading required package: iterators



In [2]:
library(data.table)
library(parallel)
library(stringr)

df <- fread("12-OUT_matched_SNP_meth_cov_outputs.csv")

df <- df[!grepl("empty", df$path), ]

df$tag <- paste0(df$region, "_", df$population, "_", df$Chr, "_", df$chunk_start, "_", df$chunk_end)

library(stringr)

df$path_unmodified <- df$path

df$path <- gsub("-caud", "", df$path)
df$path <- gsub("-hippo", "", df$path)
df$path <- gsub("-dlpfc", "", df$path)

datetime <- str_split_fixed(df$path, "allcorepera-", 2)[,2]
datetime <- gsub("\\.rds", "", datetime)
datetime <- str_split_fixed(datetime, "-", 2)

In [3]:
df$date <- as.numeric(datetime[, 1])
df$time <- as.numeric(datetime[, 2])

In [4]:
df[, datetime := as.POSIXct(paste0(date, " ", time), format = "%Y%m%d %H%M%S")]

In [5]:
# Order the data by datetime in descending order
setorder(df, region, Chr, population, chunk_start, chunk_end, -datetime)

# Remove duplicate groups, keeping the first (most recent) entry
df <- df[, .SD[1], by = .(region, Chr, population, chunk_start, chunk_end)]



In [6]:
head(df)

region,Chr,population,chunk_start,chunk_end,SNP_data,methylation_data,last_meth_value_with_SNP_coverage,first_meth_value_with_SNP_coverage,last_meth_index_with_SNP_coverage,first_meth_index_with_SNP_coverage,cov_file,modified_methylation_data,path,tag,path_unmodified,date,time,datetime
<chr>,<int>,<chr>,<int>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dttm>
caud,1,AA,8982,28981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,8982,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_caud.csv,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_8982-28981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240510-121942.rds,caud_AA_1_8982_28981,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-caud-20240510-121942.rds,20240510,121942,2024-05-10 12:19:42
caud,1,AA,28982,48981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,8982,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_caud.csv,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_28982-48981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240510-122045.rds,caud_AA_1_28982_48981,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-caud-20240510-122045.rds,20240510,122045,2024-05-10 12:20:45
caud,1,AA,48982,68981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,8982,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_caud.csv,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_48982-68981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240510-122147.rds,caud_AA_1_48982_68981,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-caud-20240510-122147.rds,20240510,122147,2024-05-10 12:21:47
caud,1,AA,68982,88981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,8982,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_caud.csv,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_68982-88981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-20240510-122249.rds,caud_AA_1_68982_88981,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-caud-20240510-122249.rds,20240510,122249,2024-05-10 12:22:49
caud,1,AA,88982,108981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,8982,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_caud.csv,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_88982-108981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-20240510-122350.rds,caud_AA_1_88982_108981,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-caud-20240510-122350.rds,20240510,122350,2024-05-10 12:23:50
caud,1,AA,108982,128981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,8982,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_caud.csv,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_108982-128981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1

In [7]:
fwrite(df, "37-OUT/37-OUT_result_table_no_dups.csv")

In [8]:
traits <- c("bp", "scz", "mdd")
for(i in 1:length(traits)){
    this_trait <- traits[i]

    dir1 <- "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud/"
    dir2 <- "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_dlpfc/"
    dir3 <- "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_hippo/"
    # List of files
    data_files.1 <- list.files(dir1, full.names = TRUE, pattern = paste0("gwas_stat_",this_trait,"_alleleprocessed_a3"))
    data_files.2 <- list.files(dir2, full.names = TRUE, pattern = paste0("gwas_stat_",this_trait,"_alleleprocessed_a3"))
    data_files.3 <- list.files(dir3, full.names = TRUE, pattern = paste0("gwas_stat_",this_trait,"_alleleprocessed_a3"))
    data_files <- c(data_files.1, data_files.2, data_files.3)
    data_files <- data_files[grepl("101324", data_files)]
    data_files <- data_files[grepl("a3", data_files)]
    data_files <- data_files[grepl("csv", data_files)]
    data_files_alt_name <- gsub("-caud", "", data_files)
    data_files_alt_name <- gsub("-dlpfc", "", data_files_alt_name)
    data_files_alt_name <- gsub("-hippo", "", data_files_alt_name)
    #data_files_alt_name <- data_files_alt_name[grepl("101324", data_files_alt_name)]
    # Split the file paths based on '-' and '_', keeping all parts
    result_table <- data.table(str_split_fixed(data_files_alt_name, "[-_]", n = Inf))
    
    result_table[, c(1:3, 5, 8:11, 14:16, 19:20, 22:25)] <- NULL
    
    
    colnames(result_table) <- c("region", "Chr", "population", "chunk_start", "chunk_end", "date", "time", "trait")
    result_table$chunk_start <- as.numeric(as.character(result_table$chunk_start))
    result_table$chunk_end <- as.numeric(as.character(result_table$chunk_end))
    these_paths <- as.data.frame(data_files)
    colnames(these_paths) <- paste0("result_", this_trait)
result_table <- cbind(result_table, these_paths)

    result_table$Chr <- as.numeric(gsub("chr", "", result_table$Chr))
    result_table$region <- gsub("//libd", "", result_table$region)
    result_table$tag <- paste0(result_table$region, "_", result_table$population, "_", result_table$Chr, "_", result_table$chunk_start, "_", result_table$chunk_end, "_", result_table$trait)
    head(result_table)
    # region	Chr	population	chunk_start	chunk_end	date	time	trait	result_bp	tag
    # <chr>	<dbl>	<chr>	<dbl>	<dbl>	<chr>	<chr>	<chr>	<chr>	<chr>
    # caud	1	AA	1008982	1028981	20240416	154932	bp	/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240416-154932_gwas_stat_bp_alleleprocessed_a3_results-101324.csv	caud_AA__1008982_1028981_bp
    # caud	1	AA	1008982	1028981	20240510	131051	bp	/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-caud-20240510-131051_gwas_stat_bp_alleleprocessed_a3_results-101324.csv	caud_AA__1008982_1028981_bp
    # caud	1	AA	1028982	1048981	20240415	140831	bp	/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240415-140831_gwas_stat_bp_alleleprocessed_a3_results-101324.csv	caud_AA__1028982_1048981_bp
    # caud	1	AA	1028982	1048981	20240510	131153	bp	/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-caud-20240510-131153_gwas_stat_bp_alleleprocessed_a3_results-101324.csv	caud_AA__1028982_1048981_bp
    # caud	1	AA	1048982	1068981	20240415	141231	bp	/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240415-141231_gwas_stat_bp_alleleprocessed_a3_results-101324.csv	caud_AA__1048982_1068981_bp
    # caud	1	AA	1048982	1068981	20240510	131254	bp	/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-caud-20240510-131254_gwas_stat_bp_alleleprocessed_a3_results-101324.csv	caud_AA__1048982_1068981_bp
    
    
    # for each combo of region x Chr x population x chunk_start x chunk_end, I only want the row with the most recent date, time
    
    # prove to me that it works by demonstrating an example to the console (and don't flood my console)
    
    # Convert 'date' and 'time' to a single POSIXct datetime column for accurate comparison
    result_table[, datetime := as.POSIXct(paste0(date, " ", time), format = "%Y%m%d %H%M%S")]
    
    # Order the data by datetime in descending order
    setorder(result_table, region, Chr, population, chunk_start, chunk_end, -datetime)
    
    # Remove duplicate groups, keeping the first (most recent) entry
    
    result_table <- result_table[, .SD[1], by = .(region, Chr, population, chunk_start, chunk_end)]
    

    df <- df[!grepl("empty", df$path), ]
    result_table$trait <- result_table$date <- result_table$time <- result_table$tag <- result_table$datetime <- NULL
    df <- merge(df, result_table, by = c("region", "Chr", "population", "chunk_start", "chunk_end"))

}

In [9]:
result_table <- df

In [10]:
head(result_table)

region,Chr,population,chunk_start,chunk_end,SNP_data,methylation_data,last_meth_value_with_SNP_coverage,first_meth_value_with_SNP_coverage,last_meth_index_with_SNP_coverage,⋯,modified_methylation_data,path,tag,path_unmodified,date,time,datetime,result_bp,result_scz,result_mdd
<chr>,<int>,<chr>,<int>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dttm>,<chr>,<chr>,<chr>
caud,1,AA,8982,28981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,⋯,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_8982-28981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240510-121942.rds,caud_AA_1_8982_28981,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-caud-20240510-121942.rds,20240510,121942,2024-05-10 12:19:42,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-caud-20240510-121942_gwas_stat_bp_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-caud-20240510-121942_gwas_stat_scz_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-caud-20240510-121942_gwas_stat_mdd_alleleprocessed_a3_results-101324.csv
caud,1,AA,28982,48981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,⋯,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_28982-48981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240510-122045.rds,caud_AA_1_28982_48981,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-caud-20240510-122045.rds,20240510,122045,2024-05-10 12:20:45,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-caud-20240510-122045_gwas_stat_bp_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-caud-20240510-122045_gwas_stat_scz_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-caud-20240510-122045_gwas_stat_mdd_alleleprocessed_a3_results-101324.csv
caud,1,AA,48982,68981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,⋯,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_48982-68981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240510-122147.rds,caud_AA_1_48982_68981,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-caud-20240510-122147.rds,20240510,122147,2024-05-10 12:21:47,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-caud-20240510-122147_gwas_stat_bp_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-caud-20240510-122147_gwas_stat_scz_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mw

In [11]:
dim(result_table)

[1] 11421    22

In [12]:
fwrite(result_table, "37-OUT/37-OUT_result_table_no_dups.csv")

In [13]:
head(result_table)

region,Chr,population,chunk_start,chunk_end,SNP_data,methylation_data,last_meth_value_with_SNP_coverage,first_meth_value_with_SNP_coverage,last_meth_index_with_SNP_coverage,⋯,modified_methylation_data,path,tag,path_unmodified,date,time,datetime,result_bp,result_scz,result_mdd
<chr>,<int>,<chr>,<int>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dttm>,<chr>,<chr>,<chr>
caud,1,AA,8982,28981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,⋯,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_8982-28981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240510-121942.rds,caud_AA_1_8982_28981,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-caud-20240510-121942.rds,20240510,121942,2024-05-10 12:19:42,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-caud-20240510-121942_gwas_stat_bp_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-caud-20240510-121942_gwas_stat_scz_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-caud-20240510-121942_gwas_stat_mdd_alleleprocessed_a3_results-101324.csv
caud,1,AA,28982,48981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,⋯,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_28982-48981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240510-122045.rds,caud_AA_1_28982_48981,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-caud-20240510-122045.rds,20240510,122045,2024-05-10 12:20:45,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-caud-20240510-122045_gwas_stat_bp_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-caud-20240510-122045_gwas_stat_scz_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-caud-20240510-122045_gwas_stat_mdd_alleleprocessed_a3_results-101324.csv
caud,1,AA,48982,68981,/dcs04/lieber/statsgen/shizhong/michael/mwas/gwas//libd_chr1.pgen,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,248918358,1069461,2202702,⋯,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_48982-68981.rds,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240510-122147.rds,caud_AA_1_48982_68981,..//output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-caud-20240510-122147.rds,20240510,122147,2024-05-10 12:21:47,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-caud-20240510-122147_gwas_stat_bp_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_caud//libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-caud-20240510-122147_gwas_stat_scz_alleleprocessed_a3_results-101324.csv,/expanse/lustre/projects/jhu152/naglemi/mw

In [14]:
### fast version modified to save combined_cor and comment out subsequent steps

# Load necessary libraries
library(data.table)
library(parallel)
library(qqman)
library(ggplot2)
library(stringr)
library(foreach)
library(doParallel)

# Define variables
desired_subpopulations <- c("AA", "EA", "all")
desired_regions <- c("caud", "hippo", "dlpfc")
traits <- c("scz", "bp", "mdd")

# Detect the number of cores and set up the parallel backend
num_cores <- 32  # Use one less than the total number of cores
cl <- makeCluster(num_cores)
registerDoParallel(cl)

# Create output directory once
dir.create("37-OUT", showWarnings = FALSE)

# Loop over subpopulations, regions, and traits
for (desired_subpopulation in desired_subpopulations) {
  for (desired_region in desired_regions) {
    for (trait in traits) {
      
      sub_df <- result_table[which(result_table$population == desired_subpopulation &
                                   result_table$region == desired_region)]
      
      if(nrow(sub_df) == 0) stop("No rows in sub_df. Error.")
       
      # Prepare variables for parallel processing
      data_files_subset <- sub_df[[paste0("result_", trait)]]
      result_table_chr <- sub_df$Chr
      
      # Parallel data reading and processing
      data_list <- foreach(i = seq_along(data_files_subset), .packages = 'data.table') %dopar% {
        dt <- fread(data_files_subset[i], showProgress = FALSE)
        dt[, chr := result_table_chr[i]]  # Add 'chr' column efficiently
        dt
      }

      dir <- unique(dirname(data_files_subset))
      
      # Combine the data tables into one
      combined_data <- rbindlist(data_list, use.names = TRUE, fill = TRUE)
      #stop("debug here")
      
      # Read and combine correlation files in parallel - This part is messed up. We should only read in cor files for corresponding sub_df paths modified with gsub
      cor_files <- str_split_fixed(data_files_subset, "_gwas", 2)[, 1]
      cor_files <- unique(cor_files)
      cor_files <- paste0(cor_files, "_cv-eval_combined.csv")

      data_files_subset <- result_table_chr <- NULL
 
      cor_list <- foreach(file = cor_files, .packages = 'data.table') %dopar% {
        fread(file, showProgress = FALSE)
      }
      
      combined_cor <- rbindlist(cor_list, use.names = TRUE, fill = TRUE)

      combined_cor <- unique(combined_cor)
      
      # Write out the combined_cor data frame with appropriate filename
      fwrite(combined_cor, file.path("37-OUT", paste0("combined_cor_", trait, "_results_a6_", desired_subpopulation, "_", desired_region, ".csv")))
      
      # Commenting out the rest of the processing steps
      # 
      # # Ensure consistent column names for merging
      # setnames(combined_data, old = "bp", new = "cg", skip_absent = TRUE)
      # setnames(combined_cor, old = "V4", new = "cg", skip_absent = TRUE)
      # 
      # # Set keys for efficient merging
      # setkey(combined_data, cg)
      # setkey(combined_cor, cg)
      # 
      # # Merge datasets
      # merged <- merge(combined_data, combined_cor, by = c("chr", "cg"))
      # 
      # combined_data <- combined_cor <- NULL
      #   
      # # Filter merged data
      # merged <- merged[cor >= 0.1]
      # merged_sample <- merged[sample(1:nrow(merged), 10000), ]
      # merged_subset <- merged[p <= 0.0001]
      # merged_subset[, SNP := paste0(chr, ":", cg)]
      # 
      # # Bonferroni correction
      # bonferroni_threshold <- 0.05 / nrow(merged)
      # 
      # bonferroni_pass <- merged[p <= bonferroni_threshold]
      # fwrite(merged, file.path("37-OUT", paste0("unfiltered_", trait, "_results_a6_", desired_subpopulation, "_", desired_region, ".csv")))
      # merged <- NULL
      # 
      # # Save filtered and unfiltered data
      # fwrite(merged_subset, file.path("37-OUT", paste0("filtered_", trait, "_results_a6_", desired_subpopulation, "_", desired_region, ".csv")))
      # fwrite(bonferroni_pass, file.path("37-OUT", paste0("filtered-bonf_", trait, "_results_a6_", desired_subpopulation, "_", desired_region, ".csv")))
      # 
      # # Save Manhattan plot
      # png(filename = file.path("37-OUT", paste0("manhattan_", trait, "_", desired_subpopulation, "_", desired_region, ".png")))
      # manhattan(merged_subset, chr = "chr", bp = "cg", p = "p", main = paste("Manhattan plot for", trait, desired_subpopulation, desired_region))
      # dev.off()
      # 
      # # Create and save QQ plot
      # png(filename = file.path("37-OUT", paste0("qqplot_", trait, "_", desired_subpopulation, "_", desired_region, ".png")))
      # qq(merged_sample$p, main = paste("QQ plot for 10k samples from", trait, desired_subpopulation, desired_region))
      # dev.off()
      # 
      # gc()
    }
  }
}

# Stop the cluster
stopCluster(cl)

In [ ]:
### fast version

# Load necessary libraries
library(data.table)
library(parallel)
library(qqman)
library(ggplot2)
library(stringr)
library(foreach)
library(doParallel)

# Define variables
desired_subpopulations <- c("AA", "EA", "all")
desired_regions <- c("caud", "hippo", "dlpfc")
traits <- c("scz", "bp", "mdd")

# Detect the number of cores and set up the parallel backend
num_cores <- 32  # Use one less than the total number of cores
cl <- makeCluster(num_cores)
registerDoParallel(cl)

# Create output directory once
dir.create("37-OUT", showWarnings = FALSE)

# Loop over subpopulations, regions, and traits
for (desired_subpopulation in desired_subpopulations) {
  for (desired_region in desired_regions) {
    for (trait in traits) {
      
      sub_df <- result_table[which(result_table$population == desired_subpopulation &
                                   result_table$region == desired_region)]

      if(nrow(sub_df) == 0) stop("No rows in sub_df. Error.")
       
      # Prepare variables for parallel processing
      data_files_subset <- sub_df[[paste0("result_", trait)]]
      result_table_chr <- sub_df$Chr
      
      # Parallel data reading and processing
      data_list <- foreach(i = seq_along(data_files_subset), .packages = 'data.table') %dopar% {
        dt <- fread(data_files_subset[i], showProgress = FALSE)
        dt[, chr := result_table_chr[i]]  # Add 'chr' column efficiently
        dt
      }

      dir <- unique(dirname(data_files_subset))
      
      # Combine the data tables into one
      combined_data <- rbindlist(data_list, use.names = TRUE, fill = TRUE)
      #stop("debug here")
      
      # Read and combine correlation files in parallel - This part is messed up. We should only read in cor files for corresponding sub_df paths modiffied with gsub
      cor_files <- str_split_fixed(data_files_subset, "_gwas", 2)[, 1]
      cor_files <- unique(cor_files)
      cor_files <- paste0(cor_files, "_cv-eval_combined.csv")

      data_files_subset <- result_table_chr <- NULL
 
      cor_list <- foreach(file = cor_files, .packages = 'data.table') %dopar% {
        fread(file, showProgress = FALSE)
      }
      
      combined_cor <- rbindlist(cor_list, use.names = TRUE, fill = TRUE)

      combined_cor <- unique(combined_cor)
      
      # Ensure consistent column names for merging
      setnames(combined_data, old = "bp", new = "cg", skip_absent = TRUE)
      setnames(combined_cor, old = "V4", new = "cg", skip_absent = TRUE)
      
      # Set keys for efficient merging
      setkey(combined_data, cg)
      setkey(combined_cor, cg)
      
      # Merge datasets
      merged <- merge(combined_data, combined_cor, by = c("chr", "cg"))

      combined_data <- combined_cor <- NULL
        
      # Filter merged data
      merged <- merged[cor >= 0.1]
      merged_sample <- merged[sample(1:nrow(merged), 10000), ]
      merged_subset <- merged[p <= 0.0001]
      merged_subset[, SNP := paste0(chr, ":", cg)]
      
      # Bonferroni correction
      bonferroni_threshold <- 0.05 / nrow(merged)

      bonferroni_pass <- merged[p <= bonferroni_threshold]
      fwrite(merged, file.path("37-OUT", paste0("unfiltered_", trait, "_results_a6_", desired_subpopulation, "_", desired_region, ".csv")))
      merged <- NULL
      
      # Save filtered and unfiltered data
      fwrite(merged_subset, file.path("37-OUT", paste0("filtered_", trait, "_results_a6_", desired_subpopulation, "_", desired_region, ".csv")))
      fwrite(bonferroni_pass, file.path("37-OUT", paste0("filtered-bonf_", trait, "_results_a6_", desired_subpopulation, "_", desired_region, ".csv")))
      
      # Save Manhattan plot
      png(filename = file.path("37-OUT", paste0("manhattan_", trait, "_", desired_subpopulation, "_", desired_region, ".png")))
      manhattan(merged_subset, chr = "chr", bp = "cg", p = "p", main = paste("Manhattan plot for", trait, desired_subpopulation, desired_region))
      dev.off()
      
      # Create and save QQ plot
      png(filename = file.path("37-OUT", paste0("qqplot_", trait, "_", desired_subpopulation, "_", desired_region, ".png")))
      qq(merged_sample$p, main = paste("QQ plot for 10k samples from", trait, desired_subpopulation, desired_region))
      dev.off()

      gc()
    }
  }
}

# Stop the cluster
stopCluster(cl)

# What's up with duplicate entries?

Identify them

In [ ]:
# version with logging so we can see where duplicates are coming from

# Load necessary libraries
library(data.table)
library(parallel)
library(qqman)
library(ggplot2)
library(stringr)
library(foreach)
library(doParallel)

# Define variables
desired_subpopulations <- c("AA", "EA", "all")
desired_regions <- c("caud", "hippo", "dlpfc")
traits <- c("scz", "bp", "mdd")

# Detect the number of cores and set up the parallel backend
num_cores <- 32  # Use one less than the total number of cores
cl <- makeCluster(num_cores)
registerDoParallel(cl)

# Create output directory once
dir.create("37-OUT", showWarnings = FALSE)

# Loop over subpopulations, regions, and traits
for (desired_subpopulation in desired_subpopulations) {
  for (desired_region in desired_regions) {
    for (trait in traits) {
      
      sub_df <- result_table[which(result_table$population == desired_subpopulation &
                                   result_table$region == desired_region)]

      if(nrow(sub_df) == 0) stop("No rows in sub_df. Error.")
       
      # Prepare variables for parallel processing
      data_files_subset <- sub_df[[paste0("result_", trait)]]
      result_table_chr <- sub_df$Chr
      
      # Parallel data reading and processing
      data_list <- foreach(i = seq_along(data_files_subset), .packages = 'data.table') %dopar% {
        dt <- fread(data_files_subset[i], showProgress = FALSE)
        dt[, chr := result_table_chr[i]]  # Add 'chr' column efficiently
        dt[, source_file := data_files_subset[i]]  # Add source file column
        dt
      }

      dir <- unique(dirname(data_files_subset))
      
      # Combine the data tables into one
      combined_data <- rbindlist(data_list, use.names = TRUE, fill = TRUE)
      #stop("debug here")
      
      # Read and combine correlation files in parallel
      cor_files <- str_split_fixed(data_files_subset, "_gwas", 2)[, 1]
      cor_files <- unique(cor_files)
      cor_files <- paste0(cor_files, "_cv-eval_combined.csv")

      data_files_subset <- result_table_chr <- NULL
 
      cor_list <- foreach(file = cor_files, .packages = 'data.table') %dopar% {
        dt <- fread(file, showProgress = FALSE)
        dt[, source_file := file]  # Add source file column
        dt
      }
      
      combined_cor <- rbindlist(cor_list, use.names = TRUE, fill = TRUE)

      combined_cor <- unique(combined_cor)
      
      # Ensure consistent column names for merging
      setnames(combined_data, old = "bp", new = "cg", skip_absent = TRUE)
      setnames(combined_cor, old = "V4", new = "cg", skip_absent = TRUE)
      
      # Set keys for efficient merging
      setkey(combined_data, chr, cg)
      setkey(combined_cor, chr, cg)

      # Check for duplicates in combined_data
      dup_combined_data <- combined_data[duplicated(combined_data, by = c("chr", "cg"))]
      if (nrow(dup_combined_data) > 0) {
          message("Duplicates found in combined_data.")
      }

      # Check for duplicates in combined_cor
      dup_combined_cor <- combined_cor[duplicated(combined_cor, by = c("chr", "cg"))]
      if (nrow(dup_combined_cor) > 0) {
        message("Duplicates found in combined_cor.")
      }
      
      # Merge datasets
      merged <- merge(combined_data, combined_cor, by = c("chr", "cg"), suffixes = c(".data", ".cor"))

      combined_data <- combined_cor <- NULL
        
      # Logging duplicates
      dup_rows <- merged[duplicated(merged, by = c("chr", "cg")) | duplicated(merged, by = c("chr", "cg"), fromLast = TRUE)]
      if (nrow(dup_rows) > 0) {
        log_file <- file.path("37-OUT", paste0("duplicate_log_", trait, "_", desired_subpopulation, "_", desired_region, ".txt"))
        fwrite(head(dup_rows, 1000), log_file)  # Limit to first 1000 duplicates
      }
        
      # Filter merged data
      merged <- merged[cor >= 0.1]
      merged_sample <- merged[sample(1:nrow(merged), min(10000, nrow(merged))), ]
      merged_subset <- merged[p <= 0.0001]
      merged_subset[, SNP := paste0(chr, ":", cg)]
      
      # Bonferroni correction
      bonferroni_threshold <- 0.05 / nrow(merged)

      bonferroni_pass <- merged[p <= bonferroni_threshold]
      fwrite(merged, file.path("37-OUT", paste0("unfiltered_", trait, "_results_a6_", desired_subpopulation, "_", desired_region, ".csv")))
      merged <- NULL
      
      # Save filtered and unfiltered data
      fwrite(merged_subset, file.path("37-OUT", paste0("filtered_", trait, "_results_a6_", desired_subpopulation, "_", desired_region, ".csv")))
      fwrite(bonferroni_pass, file.path("37-OUT", paste0("filtered-bonf_", trait, "_results_a6_", desired_subpopulation, "_", desired_region, ".csv")))
      
      # Save Manhattan plot
      png(filename = file.path("37-OUT", paste0("manhattan_", trait, "_", desired_subpopulation, "_", desired_region, ".png")))
      manhattan(merged_subset, chr = "chr", bp = "cg", p = "p", main = paste("Manhattan plot for", trait, desired_subpopulation, desired_region))
      dev.off()
      
      # Create and save QQ plot
      png(filename = file.path("37-OUT", paste0("qqplot_", trait, "_", desired_subpopulation, "_", desired_region, ".png")))
      qq(merged_sample$p, main = paste("QQ plot for 10k samples from", trait, desired_subpopulation, desired_region))
      dev.off()

      gc()
    }
  }
}

# Stop the cluster
stopCluster(cl)

### investigate further 

In [ ]:
is.data.frame(sub_df)

In [ ]:
is.data.table(sub_df)

In [ ]:
data_files_subset

In [ ]:
dir

In [ ]:
desired_subpopulation

In [ ]:
desired_region

In [ ]:
head(result_table)

In [ ]:
levels(factor(result_table$region))

In [ ]:
head(combined_data)

In [ ]:
head(combined_cor)

In [ ]:
head(sub_df)

In [ ]:
head(data_files_subset)

In [ ]:
desired_subpopulation

In [ ]:
desired_region

In [ ]:
trait

In [ ]:
head(result_table)

In [ ]:
gead()

In [ ]:
sub_df[, which(colnames(sub_df) == paste0("result_", trait))]

In [ ]:
sub_df

In [ ]:
head(combined_data)

In [ ]:
head(combined_cor)

In [ ]:
# ## slow

# # Load necessary libraries
# library(data.table)
# library(parallel)
# library(stringr)

# # Define variables
# desired_subpopulations <- c("AA", "EA", "all")
# desired_regions <- c("caud", "hippo", "dlpfc")
# traits <- c("scz", "bp", "mdd")

# num_cores <- 6
# chromosome_stats <- list()  # Store stats for each combination

# # Loop over subpopulations, regions, and traits
# for (desired_subpopulation in desired_subpopulations) {
#   for (desired_region in desired_regions) {
#     for (trait in traits) {
      
#       # Define directory path
#       dir <- paste0("/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_dt_", desired_region, "/")
      
#       # List and filter data files
#       data_files <- list.files(dir, full.names = TRUE, pattern = paste0("gwas_stat_", trait, "_alleleprocessed_a3"))
#       data_files <- data_files[grepl("csv", data_files)]
#       data_files_alt_name <- gsub(paste0("-", desired_region), "", data_files)
      
#       # Create result table
#       result_table <- data.table(str_split_fixed(data_files_alt_name, "[-_]", n = Inf))
#       result_table$path <- data_files
#       result_table$chr <- as.numeric(gsub("chr", "", result_table$V5))
#       result_table <- result_table[which(result_table$V7 == desired_subpopulation)]
      
#       # Filter by most recent date
#       result_table$region <- str_split_fixed(result_table$path, "EXPANSE_dt_", 2)[, 2]
#       result_table$region <- str_split_fixed(result_table$region, "/", 2)[, 1]
    
#       result_table[, V17 := as.Date(V17, format="%Y%m%d")]
    
#       # Filter to keep only the row with the most recent date for each V12
    
#       result_table$tag <- paste0(result_table$V11, "_", result_table$V21, "_", result_table$region, "_", result_table$chr, "_", result_table$V12)
#       resuilt_table <- result_table[which(result_table$V23 == "a3"), ]
      
#       # Sanity check for duplicates
#       dupes_check <- result_table[, .N, by=.(V12, V17)][N > 1]
#       if (nrow(dupes_check) > 0) {
#         stop("Sanity check failed: Multiple datasets found for the same V12 on the same date.")
#       }
      
#       # Combine data files
#       data_files <- result_table$path
#       data_list <- list()
#       for (i in seq_along(data_files)) {
#         data_list[[i]] <- fread(data_files[i])
#         data_list[[i]]$chr <- result_table[i]$chr
#       }
#       combined_data <- rbindlist(data_list)
      
#       # Combine correlation files
#       cor_files <- list.files(dir, full.names = TRUE, pattern = "eval")
#       cor_list <- mclapply(cor_files, fread, mc.cores = num_cores)
#       combined_cor <- rbindlist(cor_list)
      
#       # Merge datasets
#       colnames(combined_data)[4] <- "cg"
#       merged <- merge(combined_data, combined_cor)
#       merged <- merged[which(merged$cor >= 0.1), ]
#       merged_subset <- merged[which(merged$p <= 0.0001), ]
#       merged_subset$SNP <- paste0(merged_subset$chr, ":", merged_subset$cg)
      
#       # Bonferroni correction
#       bonferroni_threshold <- 0.05 / nrow(merged)
#       bonferroni_pass <- merged[merged$p <= bonferroni_threshold, ]
      
#       # Count rows by chromosome
#       unfiltered_counts <- merged[, .N, by = chr]
#       filtered_counts <- merged_subset[, .N, by = chr]
      
#       # Add to stats list
#       chromosome_stats[[paste0(trait, "_", desired_subpopulation, "_", desired_region)]] <- list(
#         unfiltered = unfiltered_counts,
#         filtered = filtered_counts,
#         bonferroni = bonferroni_pass[, .N, by = chr]
#       )
      
#       # Print counts to console
#       cat("Chromosome counts for trait:", trait, ", subpopulation:", desired_subpopulation, ", region:", desired_region, "\n")
#       cat("Unfiltered counts:\n")
#       print(unfiltered_counts)
#       cat("Filtered counts (p <= 0.01):\n")
#       print(filtered_counts)
#       cat("Bonferroni-filtered counts:\n")
#       print(chromosome_stats[[paste0(trait, "_", desired_subpopulation, "_", desired_region)]]$bonferroni)
#       gc()
#     }
#   }
# }

# # Write CSV files for stats
# for (name in names(chromosome_stats)) {
#   write.csv(cbind("Chromosome" = chromosome_stats[[name]]$unfiltered$chr, 
#                   "Unfiltered_Count" = chromosome_stats[[name]]$unfiltered$N, 
#                   "Filtered_Count" = chromosome_stats[[name]]$filtered$N, 
#                   "Bonferroni_Count" = chromosome_stats[[name]]$bonferroni$N), 
#             file = paste0(name, "_chromosome_counts.csv"), row.names = FALSE)
# }

In [ ]:
head(merged)

In [ ]:
dim(merged)

In [ ]:
merged_subset[which(merged_subset$`chr.x` != merged_subset$`chr.y`), ]